In [1]:
import os
import logging
import argparse
import json
import datetime
from typing import Union

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam
from torch.optim import Optimizer
                              

from utils import *
from losses import PSNR
from SynthTrainer import SynthTrainer
import gan

from synth_data_source import loadData

We are going to start by preparing our various models. First we will start with the model described in our target paper "Desmoking Laparoscopy Surgery Images Using an Image-to-Image Translation Guided by an Embedded Dark Channel"
We will have to define a discriminator and a generator. Note the tables here reflect processing a 512x512 image as opposed to 256x256 as the original paper used. Also note that upon inspecting the paper repository, the tables in the paper appear inaccurate. I recreated these nets to be true to their repository by looking at the logic that generates the nets used.

Since the arguments used were not specified in the paper I made a best estimate related to optional arguments based on what I thought would make sense. I also made some modification on where I apply dropout based on some advice from Dr. Florian Richter. It appears dropout was applied on all the 'decoder' layers in the unet, but it equally well could have not been applied at all depending on arguments passed at runtime. My assumption is dropout will be helpful and should be applied at the innermost layers of the decoder.

ADD TABLES HERE AS IMAGES. KATEX DOES NOT IMPLEMENT TABULAR...

I am only showing this discriminator here. Please see models.py for all the models we will be using. They take up a lot of screen space since I wrote them out explicitly.

In [2]:
# DEFINE THE PAPER DISCRIMINATOR
class Discriminator_demo(torch.nn.Module):
    def __init__(self, input_channels:int = 3):
        super().__init__()

        self.sequence = [torch.nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1), 
                    torch.nn.LeakyReLU(0.2, True)]
        
        sequence += [torch.nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), 
                     torch.nn.BatchNorm2d(128),
                     torch.nn.LeakyReLU(0.2, True)]
        
        sequence += [torch.nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), 
                     torch.nn.BatchNorm2d(256),
                     torch.nn.LeakyReLU(0.2, True)]
        
        sequence += [torch.nn.ZeroPad2d(2)]
        
        sequence += [torch.nn.Conv2d(256, 512, kernel_size=4, stride=1), 
                     torch.nn.BatchNorm2d(512),
                     torch.nn.LeakyReLU(0.2, True)]
        
        sequence += [torch.nn.ZeroPad2d(2)]
        
        sequence += [torch.nn.Conv2d(512, 1, kernel_size=4, stride=1)]
        
        self.model = torch.nn.Sequential(*sequence)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.model(x)



Now we import models. Note that UNET is the paper implementation and UNETsmolr is the ablated version with layers x-y taken out. Please reference the code and the numbers to the right of the layers. These layers have different assigned numbers than how it is organized in the report. In the code every individual nn.module has its own layer number instead of being grouped together.

In [3]:
import models

Note our trainer is located in gan.py. You will want to reference that file along with the inherited class from data_augmentation in order to understand the training, validation and testing. I have included it instead of implementing it here to reduce the length of this notebook.

You will also want to reference losses.py for PSNR and other metrics being taken for image quality.

In [4]:
#from gan import GANTrainer
from losses import PSNR
from losses import SSIMLoss
from torchmetrics import UniversalImageQualityIndex
from libs.pytorch_fsim import fsim


Now we will setup some arguments which we will use to run our trainer.

In [5]:
args = {      
        'save' : 'C:/Users/Karol/Documents/DL4H/runs/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')),
        'batch' : 8,
        'lr' : 0.0002,
        'epochs' : 60,
        'gpu' : 0,
        'use_dark_channel' : False,
        'load_net' : None,
        'run_val_and_test_every_steps' : 1718
}

args_data = {
        'pre_path' : 'C:/Users/Karol/Documents/DL4H',
        'cache_subfolder' : '/datasets/cholec80/cache',
        'cache_subfolder_test' : '/datasets/cholec80/cache_testset',
        'syn_smoke_subfolder' : '/datasets/cholec80/synthetic_smoke/',
        'dataset_subfolder' : '/datasets/cholec80/input_formatted',
        'dataset_subfolder_test' : '/datasets/cholec80/input_formatted_test'
}

Now let us load all our data in.

In [6]:
train_data, val_data, test_data = loadData(args_data)

KeyboardInterrupt: 

Now we setup our models and run our GAN trainer! This will train and output to tensorboard as it trains. Please set 'save' appropriately in args for log and tensorboard. This step is super easy.

For the inner workings please reference models.py for models and gan.py for the trainer.
The trainer also uses SynthTrainer.py and Trainer.py from the data_augmentation folder.

Most of the basic training is done in gan.py however lots of logging of metrics occurs in SynthTrainer.py

Trainer.py contains framework code to generically train various implementations of SynthTrainer. For the purpose of this project its best to mainly stick to gan.py for intuition on the actual models.

In [ ]:
import importlib
importlib.reload(models)

myUNET = models.UNETsmolr()
myDisc = models.Discriminator()


In [ ]:
myUNET.parameters

<bound method Module.parameters of UNETsmolr(
  (drp): Dropout(p=0.5, inplace=False)
  (sequence): ParameterList(
      (0): Object of type: Conv2d
      (1): Object of type: LeakyReLU
      (2): Object of type: Conv2d
      (3): Object of type: BatchNorm2d
      (4): Object of type: LeakyReLU
      (5): Object of type: Conv2d
      (6): Object of type: BatchNorm2d
      (7): Object of type: LeakyReLU
      (8): Object of type: Conv2d
      (9): Object of type: BatchNorm2d
      (10): Object of type: LeakyReLU
      (11): Object of type: Conv2d
      (12): Object of type: BatchNorm2d
      (13): Object of type: LeakyReLU
      (14): Object of type: Conv2d
      (15): Object of type: BatchNorm2d
      (16): Object of type: LeakyReLU
      (17): Object of type: Conv2d
      (18): Object of type: BatchNorm2d
      (19): Object of type: LeakyReLU
      (20): Object of type: Conv2d
      (21): Object of type: ReLU
      (22): Object of type: ConvTranspose2d
      (23): Object of type: Batch

In [12]:
#importlib.reload(gan)
args['save'] = 'C:/Users/Karol/Documents/DL4H/runs/smol_nodark_60ep{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
gan.run(args, myUNET, myDisc, train_data, val_data, test_data)

Epoch 1/60:  50%|█████     | 13752/27461 [04:21<03:44, 61.04batch/s]c:\Users\Karol\.conda\envs\DL4H_CS598_2\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Running Test: 100%|██████████| 290/290 [00:22<00:00, 13.09batch/s]
Epoch 1/60: 27464batch [08:41, 52.64batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.52batch/s]
Epoch 2/60: 27464batch [08:24, 54.46batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.72batch/s]
Epoch 3/60: 27464batch [07:58, 57.39batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.63batch/s]
Epoch 4/60: 27464batch [07:57, 57.47batch/s]                          
Running Test: 100%|█

In [ ]:
import torch
torch.cuda.empty_cache()



In [ ]:
dir()


In [ ]:
hold = dir()
for x in hold:
    if x != 'torch' and x != '_oh':
        xc = 'del '+x
        print(xc)
        exec(xc)
del compose_smoke_and_vid_img
del VideoLoader
del SynthTrainer
del SynthSmokeLoader

del getDatasets
del generateVideo

del models
del myDisc
del myUNET
del gan
del test_data
del val_data
del train_data
del synthDatafromVidData


Run training, validation, and test for scenario 2. This is full u-net and no dark channel.

In [18]:
#CODE TO TRAIN SCENARIO 2
myUNET2 = models.UNET()
myDisc2 = models.Discriminator()
args['save'] = 'C:/Users/Karol/Documents/DL4H/runs/full_nodark_60ep{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
gan.run(args, myUNET2, myDisc2, train_data, val_data, test_data)


Running Test: 100%|██████████| 290/290 [00:20<00:00, 13.86batch/s]
Epoch 1/60: 27464batch [08:57, 51.11batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.25batch/s]
Epoch 2/60: 27464batch [08:47, 52.06batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.65batch/s]
Epoch 3/60: 27464batch [08:38, 52.92batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.62batch/s]
Epoch 4/60: 27464batch [08:39, 52.91batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.47batch/s]
Epoch 5/60: 27464batch [08:39, 52.89batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.59batch/s]
Epoch 6/60: 27464batch [08:39, 52.87batch/s]                          
Running Test: 100%|██████████| 290/290 [00:21<00:00, 13.66batch/s]
Epoch 7/60: 27464batch [08:39, 52.85batch/s]                          
Running Test: 100%|██████████| 290

Run training, validation, and test for scenario 3. This is ablated u-net and dark channel.

In [ ]:
#CODE TO TRAIN SCENARIO 3
myUNET3 = models.UNETsmolr(input_channels=4)
myDisc3 = models.Discriminator()
args['save'] = 'C:/Users/Karol/Documents/DL4H/runs/smol_dark_60ep{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
args['use_dark_channel'] = True
gan.run(args, myUNET3, myDisc3, train_data, val_data, test_data)

Epoch 1/60:   3%|▎         | 808/27461 [00:50<08:59, 49.37batch/s] c:\Users\Karol\.conda\envs\DL4H_CS598_2\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([3, 256, 256])
torch.Size([3, 256, 256])


torch.Size([3, 256, 256])


torch.Size([3, 256, 256])


torch.Size([3, 256, 256])


...

torch.Size([3, 256, 256])


Running Validation:  65%|██████▍   | 89/137 [00:18<00:01, 29.26batch/s]

torch.Size([3, 256, 256])


Epoch 1/60:  18%|█▊        | 4808/27461 [06:25<30:17, 12.46batch/s]


AttributeError: 'tuple' object has no attribute 'tb_frame'

Run training, validation, and test for scenario 4. This is full u-net and dark channel.

In [ ]:
#CODE TO TRAIN SCENARIO 4
myUNET4 = models.UNET(input_channels=4)
myDisc4 = models.Discriminator()
args['save'] = 'C:/Users/Karol/Documents/DL4H/runs/full_dark_60ep{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
args['use_dark_channel'] = True
gan.run(args, myUNET4, myDisc4, train_data, val_data, test_data)

Now we must run testing on all 4 scenarios. Load in all the models at each epoch so we can plot metrics. (INTEGRATE THIS INTO TRAIN AND VALIDATE??)

In [ ]:
# YES?

http://localhost:6006